In [1]:
# Mounting to drive so it doesn't need my local storage
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [2]:
# Imports: tf for CNN, MaxPool, etc; os for file management; 
# numpy for array datatype/tf; nibabel for working with brain scan files;
# tqdm for progress bar on for loop
import tensorflow as tf
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
import random

In [3]:
# Setting some variables to represent info about the dataset 
# if using a different dataset, ideally all you would have to change is these variables
IMG_WIDTH = 256
IMG_HEIGHT= 256
IMG_DEPTH = 192
IMG_PATH = '/content/drive/My Drive/NFBS_Dataset'
NUM_TRAIN_EX = 110
NUM_TOTAL_EX = 125
NUM_CHANNELS = 1
C1_FEATURES = 32

In [4]:
# Pulling the unique file names of each scan's folder
train_ids = next(os.walk(IMG_PATH))[1][:NUM_TRAIN_EX]
test_ids = next(os.walk(IMG_PATH))[1][NUM_TRAIN_EX:]

In [5]:
# Initializing X_train and Y_train
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.float32)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.float32)

In [6]:
# Loading examples from data file to array

for n in tqdm(range(NUM_TRAIN_EX)): # tqdm is for the progress bar
  id = train_ids[n]
  
  trainx_path = IMG_PATH + '/' + id + '/sub-' + id + '_ses-NFB3_T1w.nii.gz'
  imgx = nib.load(trainx_path)
  X_train[n][:][:][:] = imgx.get_fdata()

  trainy_path = IMG_PATH + '/' + id + '/sub-' + id + '_ses-NFB3_T1w_brainmask.nii.gz'
  imgy = nib.load(trainy_path)
  Y_train[n][:][:][:] = imgy.get_fdata()

100%|██████████| 110/110 [01:29<00:00,  1.23it/s]


In [7]:
X_temp = np.zeros((NUM_TRAIN_EX*IMG_DEPTH, IMG_HEIGHT, IMG_WIDTH), dtype=np.float32)
Y_temp = np.zeros((NUM_TRAIN_EX*IMG_DEPTH, IMG_HEIGHT, IMG_WIDTH), dtype=np.float32)

for n in tqdm(range(NUM_TRAIN_EX)):
    for i in range(IMG_DEPTH):
      X_temp[(IMG_DEPTH*n) + i] = X_train[n, :, :, i]
      Y_temp[(IMG_DEPTH*n) + i] = Y_train[n, :, :, i]

100%|██████████| 110/110 [00:27<00:00,  4.05it/s]


In [8]:
X_train = X_temp
Y_train = Y_temp

In [9]:
# Pooling path 3D
'''
input_layer = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH, NUM_CHANNELS))

c1 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(input_layer)
d1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d1)
p1 = tf.keras.layers.MaxPool3D((2,2,2))(c1)

c2 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
d2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d2)
p2 = tf.keras.layers.MaxPool3D((2,2,2))(c2)

c3 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
d3 = tf.keras.layers.Dropout(0.1)(c3)
c3 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d3)
p3 = tf.keras.layers.MaxPool3D((2,2,2))(c3)

c4 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
d4 = tf.keras.layers.Dropout(0.1)(c4)
c4 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d4)
p4 = tf.keras.layers.MaxPool3D((2,2,2))(c4)

c5 = tf.keras.layers.Conv3D(C1_FEATURES*16, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
d5 = tf.keras.layers.Dropout(0.1)(c5)
c5 = tf.keras.layers.Conv3D(C1_FEATURES*16, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d5)
'''

"\ninput_layer = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH, NUM_CHANNELS))\n\nc1 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(input_layer)\nd1 = tf.keras.layers.Dropout(0.1)(c1)\nc1 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d1)\np1 = tf.keras.layers.MaxPool3D((2,2,2))(c1)\n\nc2 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)\nd2 = tf.keras.layers.Dropout(0.1)(c2)\nc2 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d2)\np2 = tf.keras.layers.MaxPool3D((2,2,2))(c2)\n\nc3 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)\nd3 = tf.keras.layers.Dropout(0.1)(c3)\nc3 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', k

In [10]:
# Upsampling path 3D
'''
u6 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*8, (2,2,2), strides=(2,2,2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([c4, u6])
c6 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
d6 = tf.keras.layers.Dropout(0.1)(c6)
c6 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d6)

u7 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*4, (2,2,2), strides=(2,2,2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([c3, u7])
c7 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
d7 = tf.keras.layers.Dropout(0.1)(c7)
c7 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d7)

u8 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*2, (2,2,2), strides=(2,2,2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([c2, u8])
c8 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
d8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv3D(C1_FEATURES*2, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d8)

u9 = tf.keras.layers.Conv3DTranspose(C1_FEATURES, (2,2,2), strides=(2,2,2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([c1, u9])
c9 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
d9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv3D(C1_FEATURES, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d9)

outputs = tf.keras.layers.Conv3D(1, (1,1,1), activation='sigmoid')(c9)
'''

"\nu6 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*8, (2,2,2), strides=(2,2,2), padding='same')(c5)\nu6 = tf.keras.layers.concatenate([c4, u6])\nc6 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)\nd6 = tf.keras.layers.Dropout(0.1)(c6)\nc6 = tf.keras.layers.Conv3D(C1_FEATURES*8, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d6)\n\nu7 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*4, (2,2,2), strides=(2,2,2), padding='same')(c6)\nu7 = tf.keras.layers.concatenate([c3, u7])\nc7 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)\nd7 = tf.keras.layers.Dropout(0.1)(c7)\nc7 = tf.keras.layers.Conv3D(C1_FEATURES*4, (3,3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d7)\n\nu8 = tf.keras.layers.Conv3DTranspose(C1_FEATURES*2, (2,2,2), strides=(2,2,2), padding='same')(c7)\nu8 = tf.keras.layers.concatenate([

In [11]:
# Pooling Layer 2D

input_layer = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, NUM_CHANNELS))

c1 = tf.keras.layers.Conv2D(C1_FEATURES, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(input_layer)
c1 = tf.keras.layers.Conv2D(C1_FEATURES, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPool2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(C1_FEATURES*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
d2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(C1_FEATURES*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d2)
p2 = tf.keras.layers.MaxPool2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(C1_FEATURES*4, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Conv2D(C1_FEATURES*4, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPool2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(C1_FEATURES*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
d4 = tf.keras.layers.Dropout(0.1)(c4)
c4 = tf.keras.layers.Conv2D(C1_FEATURES*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d4)
p4 = tf.keras.layers.MaxPool2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(C1_FEATURES*16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Conv2D(C1_FEATURES*16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

In [12]:
# Upsampling Path 2D

u6 = tf.keras.layers.Conv2DTranspose(C1_FEATURES*8, (2,2), strides=(2,2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([c4, u6])
c6 = tf.keras.layers.Conv2D(C1_FEATURES*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
d6 = tf.keras.layers.Dropout(0.1)(c6)
c6 = tf.keras.layers.Conv2D(C1_FEATURES*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d6)

u7 = tf.keras.layers.Conv2DTranspose(C1_FEATURES*4, (2,2), strides=(2,2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([c3, u7])
c7 = tf.keras.layers.Conv2D(C1_FEATURES*4, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Conv2D(C1_FEATURES*4, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(C1_FEATURES*2, (2,2), strides=(2,2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([c2, u8])
c8 = tf.keras.layers.Conv2D(C1_FEATURES*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
d8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(C1_FEATURES*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(d8)

u9 = tf.keras.layers.Conv2DTranspose(C1_FEATURES, (2,2), strides=(2,2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([c1, u9])
c9 = tf.keras.layers.Conv2D(C1_FEATURES, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Conv2D(C1_FEATURES, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)

In [13]:
model = tf.keras.Model(inputs=input_layer, outputs=outputs)
model.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 32) 0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [14]:
results = model.fit(X_train, Y_train, batch_size=16, validation_split=0.2, epochs=1)

1056/1056 [==============================] - 16088s 15s/step - loss: 0.1021 - accuracy: 0.8978 - val_loss: 0.1018 - val_accuracy: 0.8982
